In [1]:
from wsp.camera.camera import BaseCamera
from wsp.utils.paths import WSP_PATH, CONFIG_PATH
from wsp.utils.utils import loadconfig
import time


class WinterDeepCamera(BaseCamera):
    
    """
    Winter Deep Camera implementation.
    This class extends BaseCamera to provide specific functionality for the Winter Deep camera.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Additional initialization if needed

config = loadconfig(CONFIG_PATH)

cam = WinterDeepCamera(base_directory = WSP_PATH,
                    config = config,
                    camname = "WINTER-deep",
                    daemon_pyro_name = "FakeCamera",
                    logger = None,
                    verbose=False)



telescope: wsp_path = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp
WINTER-deep local interface: State changed: 2025-08-22 19:12:41.890894: OFF -> READY


In [2]:
cam.print_state()

state = cam.state
print("Camera state:", state["camera_state"])

{
  "camera_state": "READY",
  "camname": "WINTER-deep",
  "is_connected": true,
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null
}
Camera state: READY


In [3]:
print("Starting up camera...")
sendtime = time.time()
cam.startupCamera()

while cam.state["camera_state"] != "READY":

    #print("Waiting for camera to be ready...")
    cam.update_state()
endtime = time.time()

print(f"Camera startup took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")


Starting up camera...
WINTER-deep local interface: State changed: 2025-08-22 19:12:46.449468: READY -> STARTUP_REQUESTED
WINTER-deep local interface: State changed: 2025-08-22 19:12:46.450743: STARTUP_REQUESTED -> READY
Camera startup took 0.0 seconds
Camera is READY.


In [8]:
cam.print_state()

WINTER-deep local interface: State verification timeout! Expected STARTUP_REQUESTED but daemon reports READY after 13.7s
{
  "camera_state": "READY",
  "camname": "WINTER-deep",
  "is_connected": true,
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null
}


In [4]:
cam.setExposure(10)
cam.print_state()

WINTER-deep local interface: State changed: 2025-08-22 19:12:49.641170: READY -> SETTING_PARAMETERS
WINTER-deep local interface: State changed: 2025-08-22 19:12:49.642248: SETTING_PARAMETERS -> READY
{
  "camera_state": "READY",
  "camname": "WINTER-deep",
  "is_connected": true,
  "imdir": "",
  "imname": "",
  "imstarttime": "",
  "imtype": null,
  "immode": null
}


In [ ]:
print("Doing exposure...")
print(f"time = {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
sendtime = time.time()
cam.doExposure()
#time.sleep(1.0)  # Wait for daemon to process
# first wait until it registers that camera_state is EXPOSING
print("Waiting for exposure to complete...")
while cam.camera_state in ("EXPOSING", "SETTING_PARAMETERS"):
    cam.update_state()
    time.sleep(0.1)
endtime = time.time()

print(f"Camera exposure took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")


Doing exposure...
time = 2025-08-22 15:12:53
WINTER-deep local interface: running doExposure
WINTER-deep local interface: State changed: 2025-08-22 19:12:53.026445: READY -> EXPOSING
WINTER-deep local interface: updating state dictionaries
WINTER-deep local interface: making FITS header
WINTER-deep local interface: sending doExposure request to camera: imdir = /Users/nlourie/data/images/20250822, imname = FakeCamera_20250822-191253-025
Waiting for camera to start exposure...
Camera State: EXPOSING
Waiting for exposure to complete...
WINTER-deep local interface: State changed: 2025-08-22 19:12:53.032862: EXPOSING -> READY
WINTER-deep local interface: State changed: 2025-08-22 19:12:53.139240: READY -> EXPOSING
WINTER-deep local interface: State changed: 2025-08-22 19:13:04.058748: EXPOSING -> READY


KeyboardInterrupt: 

In [18]:
import time
import traceback

from wsp.camera.camera import BaseCamera
from wsp.utils.paths import WSP_PATH, CONFIG_PATH
from wsp.utils.utils import loadconfig
import time


class WinterDeepCamera(BaseCamera):
    
    """
    Winter Deep Camera implementation.
    This class extends BaseCamera to provide specific functionality for the Winter Deep camera.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Additional initialization if needed

config = loadconfig(CONFIG_PATH)

cam = WinterDeepCamera(base_directory = WSP_PATH,
                    config = config,
                    camname = "WINTER-deep",
                    daemon_pyro_name = "FakeCamera",
                    logger = None,
                    verbose=False)


# Monkey patch the camera's update_state method to add debugging
def debug_update_state(cam):
    original_update_state = cam.update_state
    
    def wrapped_update_state():
        print(f"\n[DEBUG] update_state called from:")
        # Print call stack
        for line in traceback.format_stack()[:-1]:
            if "wsp" in line or "debug" in line:
                print(line.strip())
        
        print(f"[DEBUG] Current local state: {cam._camera_state.value}")
        print(f"[DEBUG] About to poll daemon...")
        
        # Call original
        original_update_state()
        
        print(f"[DEBUG] After polling - local state: {cam._camera_state.value}")
        print(f"[DEBUG] Remote state dict: {cam.remote_state.get('camera_state', 'UNKNOWN')}")
        
    cam.update_state = wrapped_update_state


# Also patch camera_state setter to see all state changes
def debug_camera_state(cam):
    original_setter = type(cam).camera_state.fset
    
    def wrapped_setter(self, new_state):
        print(f"\n[DEBUG] State setter called:")
        print(f"[DEBUG] Changing from {self._camera_state.value} to {new_state.value}")
        print(f"[DEBUG] Call stack:")
        for line in traceback.format_stack()[:-1]:
            if "wsp" in line or "debug" in line:
                print(line.strip())
        
        # Call original
        original_setter(self, new_state)
    
    # Replace the setter
    type(cam).camera_state = property(
        fget=type(cam).camera_state.fget,
        fset=wrapped_setter
    )


print("Starting up camera...")
sendtime = time.time()
cam.startupCamera()

while cam.state["camera_state"] != "READY":

    #print("Waiting for camera to be ready...")
    cam.update_state()
endtime = time.time()

print(f"Camera startup took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")


# Apply debug patches
debug_update_state(cam)
debug_camera_state(cam)

cam.setExposure(10)
while cam.state["camera_state"] != "READY":
    cam.update_state()
    time.sleep(0.1)

print("\n=== Starting exposure test ===\n")

print("Doing exposure...")
print(f"time = {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
sendtime = time.time()

print("Doing exposure...")
print(f"time = {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}")
sendtime = time.time()
cam.doExposure()
time.sleep(0.2)  # Wait for daemon to process

while cam.state["camera_state"] != "READY":
    cam.update_state()
    time.sleep(0.1)
endtime = time.time()

print(f"Camera exposure took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")


print(f"Camera exposure took {endtime - sendtime:.1f} seconds")
print("Camera is READY.")

WINTER-deep local interface: State changed: OFF -> READY
Starting up camera...
WINTER-deep local interface: State changed: READY -> STARTUP_REQUESTED
WINTER-deep local interface: State changed: STARTUP_REQUESTED -> READY
Camera startup took 0.0 seconds
Camera is READY.

[DEBUG] State setter called:
[DEBUG] Changing from READY to SETTING_PARAMETERS
[DEBUG] Call stack:
File "/Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp/camera/camera.py", line 329, in setExposure
    self.camera_state = CameraState.SETTING_PARAMETERS
WINTER-deep local interface: State changed: READY -> SETTING_PARAMETERS

[DEBUG] State setter called:
[DEBUG] Changing from SETTING_PARAMETERS to READY
[DEBUG] Call stack:
File "/Users/nlourie/Desktop/Work/MIT/WINTER/GIT/observatory/wsp/camera/camera.py", line 334, in setExposure
    self.camera_state = CameraState.READY
WINTER-deep local interface: State changed: SETTING_PARAMETERS -> READY

=== Starting exposure test ===

Doing exposure...
time = 2025-08-22 14